<h1><font color="#113D68" size=6>Deep Learning con Python y Keras</font></h1>

<h1><font color="#113D68" size=5>Parte 3. Multilayer Perceptron</font></h1>

<h1><font color="#113D68" size=4>7. Proyecto de regresión</font></h1>

<br><br>
<div style="text-align: right">
<font color="#113D68" size=3>Manuel Castillo Cara</font><br>

</div>

---

<a id="indice"></a>
<h2><font color="#004D7F" size=5>Índice</font></h2>

* [0. Contexto](#section0)
* [1. Topología de linea base](#section1)
* [2. Optimizar el rendimiento con procesamiento de datos](#section2)
* [3. Ajuste de la toplogía](#section3)
    * [3.1. Evaluar una topología más profunda](#section3.1)
    * [3.2. Evaluar una topología más grande](#section3.2)

---
<a id="section0"></a>
# <font color="#004D7F" size=6> 0. Contexto</font>

En este tutorial del proyecto, trabajaremos cómo desarrollar y evaluar modelos de redes neuronales para un problema de regresión:
* Cómo crear un modelo de red neuronal para un problema de regresión.
* Cómo utilizar scikit-learn con Keras para evaluar modelos mediante validación cruzada.
* Cómo realizar la preparación de datos para mejorar la habilidad.
* Cómo Optimizar la topología de red de modelos.

In [1]:
import tensorflow as tf
# Eliminar warning
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section1"></a>
# <font color="#004D7F" size=6>1. Topología de linea base</font>

En esta sección crearemos un modelo de red neuronal de referencia para el problema de regresión. Comencemos importando todas las funciones y objetos que necesitaremos para este tutorial.

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i>
Más información sobre el dataset [Boston House Price](http://lib.stat.cmu.edu/datasets/boston)

In [8]:
# Binary Classification with Sonar Dataset: Baseline
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

# load dataset
from google.colab import drive
drive.mount('/content/drive')
path= "/content/drive/MyDrive/CursoDeepLearning/Datasets/housing.csv"

dataframe= pd.read_csv(path,delim_whitespace=True, header=None)
dataset= dataframe.values
dataset

# split into input (X) and output (Y) variables
x= dataset[:,0:13]
y= dataset[:,13]
y

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


array([24. , 21.6, 34.7, 33.4, 36.2, 28.7, 22.9, 27.1, 16.5, 18.9, 15. ,
       18.9, 21.7, 20.4, 18.2, 19.9, 23.1, 17.5, 20.2, 18.2, 13.6, 19.6,
       15.2, 14.5, 15.6, 13.9, 16.6, 14.8, 18.4, 21. , 12.7, 14.5, 13.2,
       13.1, 13.5, 18.9, 20. , 21. , 24.7, 30.8, 34.9, 26.6, 25.3, 24.7,
       21.2, 19.3, 20. , 16.6, 14.4, 19.4, 19.7, 20.5, 25. , 23.4, 18.9,
       35.4, 24.7, 31.6, 23.3, 19.6, 18.7, 16. , 22.2, 25. , 33. , 23.5,
       19.4, 22. , 17.4, 20.9, 24.2, 21.7, 22.8, 23.4, 24.1, 21.4, 20. ,
       20.8, 21.2, 20.3, 28. , 23.9, 24.8, 22.9, 23.9, 26.6, 22.5, 22.2,
       23.6, 28.7, 22.6, 22. , 22.9, 25. , 20.6, 28.4, 21.4, 38.7, 43.8,
       33.2, 27.5, 26.5, 18.6, 19.3, 20.1, 19.5, 19.5, 20.4, 19.8, 19.4,
       21.7, 22.8, 18.8, 18.7, 18.5, 18.3, 21.2, 19.2, 20.4, 19.3, 22. ,
       20.3, 20.5, 17.3, 18.8, 21.4, 15.7, 16.2, 18. , 14.3, 19.2, 19.6,
       23. , 18.4, 15.6, 18.1, 17.4, 17.1, 13.3, 17.8, 14. , 14.4, 13.4,
       15.6, 11.8, 13.8, 15.6, 14.6, 17.8, 15.4, 21

Comencemos por definir la función que crea nuestro modelo de línea de base. 
1. Riene una sola capa oculta completamente conectada con el mismo número de neuronas que los atributos de entrada (13). 
2. La red utiliza la función de activación ReLu para la capa oculta. 
* Al ser problema de regresion no tiene funcion de activacion en la salida
3. Utiliza el algoritmo de optimización ADAM y se optimiza una función de pérdida de error cuadrático medio. 
4. El objeto Wrapper para regresión se llama `KerasRegressor`. 
5. Evaluamos este modelo de línea de base con 10-fold.

In [9]:
# define base model
def baseline_model():
  model= Sequential()
  model.add(Dense(13,input_dim=13,activation='relu'))
  model.add(Dense(1))
  # Compile model
  model.compile(loss='mean_squared_error',optimizer='Adam',metrics=['accuracy']) 
  return model  

# evaluate model
estimator= KerasRegressor(build_fn=baseline_model,epochs=100, batch_size=5, verbose=0)

kfold= KFold(n_splits=10,shuffle=True) 
results= cross_val_score(estimator, x,y, cv=kfold)
print("Accuracy base: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Accuracy base: -30.72 (5.86) MSE


---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section2"></a>
# <font color="#004D7F" size=6>2. Optimizar el rendimiento con procesamiento de datos</font>

Podemos utilizar el marco de trabajo [`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) para realizar la **estandarización** durante el proceso de evaluación del modelo, dentro de cada fold. 

In [12]:
# Libraries
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# evaluate model with standardized dataset
estimators= []
estimators.append( ('standardize',StandardScaler()) )
estimators.append( ('NN',KerasRegressor(build_fn=baseline_model,epochs=100,batch_size=5,verbose=0)) )
pipeline= Pipeline(estimators)

kfold= KFold(n_splits=10,shuffle=True) 
results= cross_val_score(pipeline, x,y, cv=kfold)
print("Accuracy optimizado (y sin normalizar): %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Accuracy optimizado: -1346.07% (613.56%)


In [14]:
import numpy as np
y_normal = (y - y.mean())/(np.max(y) - np.min(y)) # no se si esta ok

# evaluate model with standardized dataset
estimators= []
estimators.append( ('standardize',StandardScaler()) )
estimators.append( ('NN',KerasRegressor(build_fn=baseline_model,epochs=100,batch_size=5,verbose=0)) )
pipeline= Pipeline(estimators)

kfold= KFold(n_splits=10,shuffle=True) 
results= cross_val_score(pipeline, x,y_normal, cv=kfold)
print("Accuracy optimizado (y normalizado): %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Accuracy optimizado: -0.81% (0.33%)


<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i>
**Trabajo a realizar:** Una extensión adicional de esta sección sería aplicar de manera similar un cambio de escala a la variable de salida, como normalizarla al rango de 0 a 1 y usar una función de activación Sigmoide o similar en la capa de salida para reducir las predicciones de salida al mismo rango.

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section3"></a>
# <font color="#004D7F" size=6>3. Ajuste de la toplogía</font>

En esta sección evaluaremos dos topologías (más profunda y amplia) para mejorar aún más el rendimiento del modelo.

<a id="section3.1"></a>
# <font color="#004D7F" size=5>3.1. Evaluar una topología más profunda</font>

Una forma de mejorar el rendimiento de una red neuronal es agregar más capas para hacerla más profunda. En este caso con aproximadamente la mitad del número de neuronas. Nuestra topología de red ahora se ve así:
```
    13 entradas -> [13 -> 6] -> 1 salida
```
Podemos evaluar esta topología de red de la misma manera que anteriormente, al mismo tiempo que usamos la estandarización que demostró un mejor rendimiento.

In [16]:
# define the model
def larger_model():
  model= Sequential()
  model.add(Dense(13,input_dim=13,activation='relu'))
  model.add(Dense(6,activation='relu'))
  model.add(Dense(1))
  # Compile model
  model.compile(loss='mean_squared_error',optimizer='Adam',metrics=['accuracy']) 
  return model  

# evaluate model
y_normal = (y - y.mean())/(np.max(y) - np.min(y)) # no se si esta ok

estimators= []
estimators.append( ('standardize',StandardScaler()) )
estimators.append( ('Deep-NN',KerasRegressor(build_fn=larger_model,epochs=100,batch_size=5,verbose=0)) )
pipeline= Pipeline(estimators)

kfold= KFold(n_splits=10,shuffle=True) 
results= cross_val_score(pipeline, x,y_normal, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Larger: -0.01 (0.00) MSE


<a id="section3.2"></a>
# <font color="#004D7F" size=5>3.2. Evaluar una topología más grande</font>

Otro enfoque para aumentar la capacidad de representación del modelo es crear una red más amplia. Aquí, hemos aumentado el número de neuronas en la capa oculta en de 13 a 20. La topología de nuestra red más amplia se puede resumir de la siguiente manera:

```
    13 entradas -> [20] -> 1 salida
```

In [17]:
# define wider model
def wider_model():
  model= Sequential()
  model.add(Dense(20,input_dim=13,activation='relu'))
  model.add(Dense(1))
  # Compile model
  model.compile(loss='mean_squared_error',optimizer='Adam',metrics=['accuracy']) 
  return model  

# evaluate model with standardized dataset
y_normal = (y - y.mean())/(np.max(y) - np.min(y)) # no se si esta ok

estimators= []
estimators.append( ('standardize',StandardScaler()) )
estimators.append( ('Wider-NN',KerasRegressor(build_fn=wider_model,epochs=100,batch_size=5,verbose=0)) )
pipeline= Pipeline(estimators)

kfold= KFold(n_splits=10,shuffle=True) 
results= cross_val_score(pipeline, x,y_normal, cv=kfold)
print("Wider: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Wider: -0.01 (0.00) MSE


La construcción del modelo ve una caída adicional en el error a aproximadamente 22 mil dólares cuadrados. Este no es un mal resultado para este problema.

<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-coffee" aria-hidden="true" style="color:#004D7F"></i> </font></div>